In [ ]:
## 測試 MSSQL 上傳、更新資料速度

* DB：Azure SQL Database   
* Pricing Tier: S3

In [1]:
import time
import datetime
import numpy as np
import pandas as pd
from hotaidata.connectdb import ConnectedMSSQL
# engine: pyodbc, pyodbc.pooling=False

today_datetime = datetime.date.today() # 不包含
today_string_dash = today_datetime.strftime("%Y-%m-%d")

one_week_datetime = today_datetime + datetime.timedelta(days=-7)
one_week_string_dash = one_week_datetime.strftime("%Y-%m-%d")

In [ ]:
# 讀取資料

In [2]:
oneId_mapping_df = ConnectedMSSQL('oneid-hash.database.windows.net', 'oneid_hash_prod', 'jerryko').read_data("""
    SELECT OneID AS OneId, Service, MIN(CreateDate) AS CreateDate, MAX(UpdateDate) AS UpdateDate
    FROM OneIDHashMapping 
    WHERE UpdateDate BETWEEN '{0}' AND '{1}'
    GROUP BY OneID, Service
""".format(one_week_string_dash, today_string_dash))

In [3]:
new_table = oneId_mapping_df.pivot('OneId', 'Service', 'UpdateDate').reset_index()

dataframe 共 173,645 列，7 欄。  
但其中 6 欄含有大量 NULL，是一個稀疏矩陣。可能會影響到 insert 的速度。

In [6]:
new_table.shape

(173645, 7)

### insert 資料
由於會需要進行 MSSQL merge，因此先創建一個暫存表，為要更新的資料

In [8]:
tableau_db = ConnectedMSSQL('hc-media-db.database.windows.net', 'tableau', 'jerryko')
tableau_db.execute_sql("TRUNCATE TABLE OneId_LoninPlatform_temp")
st = time.time()
tableau_db.insert_into_data(new_table, 'OneId_LoninPlatform_temp')
ed = time.time()

D:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:1423: UserWarning: The provided table name 'OneId_LoninPlatform_temp' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


In [9]:
# insert 花費時間
ed - st

24.752785444259644

### update 資料
使用 MSSQL merge 方法，將有比對到 pk 的資料進行更新，更新規則為 tmpe table 的欄為有值才會更新到主表；沒有比對到 pk 的資料，則代表為新資料，將進行 insert。

In [12]:
update_query = """
    MERGE OneId_LoninPlatform AS f
    USING OneId_LoninPlatform_Temp AS t
    ON f.OneId = t.OneId
    WHEN MATCHED THEN 
        UPDATE SET 
            f.TAPP = ISNULL(t.TAPP, f.TAPP),
            f.ToyotaWeb = ISNULL(t.ToyotaWeb, f.ToyotaWeb),
            f.LAPP = ISNULL(t.LAPP, f.LAPP),
            f.LexusWeb = ISNULL(t.LexusWeb, f.LexusWeb),
            f.YoxiApp = ISNULL(t.YoxiApp, f.YoxiApp),
            f.YoxiWeb = ISNULL(t.YoxiWeb, f.YoxiWeb)
    WHEN NOT MATCHED THEN
        INSERT VALUES(t.OneId, t.TAPP, t.ToyotaWeb, t.LAPP, t.LexusWeb, t.YoxiApp, t.YoxiWeb);
"""

In [13]:
st = time.time()
tableau_db.execute_sql(update_query)
et = time.time()

主表資料量為 1,225,509；暫存表為 173,645。

In [14]:
# update 花費時間
et - st

28.036362886428833